### Required imports

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from itertools import product

In [ ]:
import warnings
#To ignore all warnings in model output
warnings.filterwarnings("ignore")

### Basic Statistics

In [ ]:
jpm = pd.read_excel('JPM Statistics and Regression Analysis.xlsx', usecols=[0,5], parse_dates=True)
jpm['return']= np.log(jpm['Adj Close']/jpm['Adj Close'].shift(1))
jpm.dropna(inplace=True)
jpm.head()

In [ ]:
average_price = np.mean(jpm['Adj Close'])
stock_volatility = np.std(jpm['return'].dropna())
average_price
stock_volatility

### Linear Regression

In [ ]:
SP = pd.read_excel('JPM Statistics and Regression Analysis.xlsx',sheet_name='Regression Analysis', usecols=[0,2], parse_dates=True)
SP['return']= np.log(SP['S&P 500 Adj Close Price (X)']/SP['S&P 500 Adj Close Price (X)'].shift(1))
SP.dropna(inplace=True)
SP.head()

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression(fit_intercept=True)

X = SP['return']
y = jpm['return']

model.fit(X[:, np.newaxis], y)

yfit = model.predict(X[:, np.newaxis])
fig, ax1 = plt.subplots()
plt.scatter(X, y)
plt.plot(X, yfit, color='red');

In [ ]:
print("Model slope:    ", model.coef_[0])
print("Model intercept:", model.intercept_)
print("R squared: ", model.score(X[:, np.newaxis],y))

### Linear Regression II

In [ ]:
import statsmodels.api as sm

In [ ]:
x = sm.add_constant(X)

In [ ]:
model = sm.OLS(y, x)

In [ ]:
results = model.fit()

In [ ]:
print(results.summary())

### Univariate Time Series

In [ ]:
home_price = pd.read_excel("CSUSHPISA.xls", skiprows=10, index_col='observation_date', parse_dates=True)
home_price = home_price.asfreq('MS')
home_price.tail()
home_price.index

In [ ]:
home_price.plot()

### Forecast S&P/Case-Shvalue_countsU.S. National Home Price Index using an ARMA model.

#### Identify the best order =`(p,q)` for our ARMA model

In [ ]:
order_aic_bic =[]
# Loop over AR order
for p in range(4):
    # Loop over MA order
    for q in range(4):
        # Fit model
        model = SARIMAX(home_price, order=(p,0,q))
        try:
            results = model.fit()
        except:
            pass
        # print the model order and the AIC/BIC values
        #print(p, q, results.aic, results.bic)
        # Add order and scores to list
        order_aic_bic.append((p, q, results.aic, results.bic))
# Make DataFrame of model order and AIC/BIC scores
order_df = pd.DataFrame(order_aic_bic, columns=['p','q', 'aic', 'bic'])

# Sort the order_df by AIC/BIC
order_df.sort_values('aic')

# Sort by BIC
order_df.sort_values('bic')

#### Select the best model with the chosen order `(p,q)`

In [ ]:
# create ARMA model
from statsmodels.tsa.arima_model import ARMA
# Instantiate model object
model = SARIMAX(home_price, order=(3,0,1))
# Fit model
results = model.fit()
results.summary()

#### Make out-of-sample predictions

In [ ]:
# Make out of sample forecast
forecast = results.get_forecast(steps=24)

#Determine mean forecast
mean_forecast = forecast.predicted_mean

mean_forecast

### Implement the Augmented Dickey-Fuller Test for checking the existence of a unit root in Case-Shiller Index series.

In [ ]:
from statsmodels.tsa.stattools import adfuller
adf = adfuller(home_price['CSUSHPISA'])
adf

In [ ]:
f'The t-value {adf[0]} is greater than the critical values {adf[4].values()} at significance levels  {adf[4].keys()} , so we cannot reject the null hypothesis of unit root'

### Implement an ARIMA(p,d,q) model. Determine p, d, q using Information Criterion or Box-Jenkins methodology. Comment the results.

##### Transform and difference of the data to make the data stationary

In [ ]:
y_stationary = np.log(home_price['CSUSHPISA']).diff().dropna()

##### Helper functions to test for stationarity

In [ ]:
#define function for kpss test
from statsmodels.tsa.stattools import kpss
#define KPSS
def kpss_test(timeseries):
    print ('Results of KPSS Test:')
    kpsstest = kpss(timeseries, regression='c')
    kpss_output = pd.Series(kpsstest[0:3], index=['Test Statistic','p-value','Lags Used'])
    for key,value in kpsstest[3].items():
        kpss_output['Critical Value (%s)'%key] = value
    return kpss_output


In [ ]:
#define function for ADF test
from statsmodels.tsa.stattools import adfuller
def adf_test(timeseries):
    #Perform Dickey-Fuller test:
    print ('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    return dfoutput

In [ ]:
kpss_test(y_stationary)
adf_test(y_stationary)

##### Both KPSS and ADFuller show that the log transform and first difference make the house price index stationary

In [ ]:
y_stationary.plot()

In [ ]:
plot_acf(y_stationary);
plot_pacf(y_stationary, alpha=.05);

#### Determine the best order `(p,d,q)`

In [ ]:
order_aic_bic =[]
# Loop over AR, Integration and MA order
for p,d,q in product(range(4), repeat=3):
    
    # Fit model
    model = SARIMAX(np.log(home_price), order=(p,d,q))
    try:
        results = model.fit()
    except:
        pass
    # print the model order and the AIC/BIC values
    #print(p, q, results.aic, results.bic)
    # Add order and scores to list
    order_aic_bic.append((p, d, q, results.aic, results.bic))
# Make DataFrame of model order and AIC/BIC scores
order_df = pd.DataFrame(order_aic_bic, columns=['p','d','q', 'aic', 'bic'])

# Sort by AIC
order_df.sort_values('aic')

# Sort by BIC
order_df.sort_values('bic')

#### Make in-sample prediction (one-step ahead)

##### The best order appear to be `order=(3,1,3)`, let's use that

In [ ]:
model = SARIMAX(np.log(home_price), order=(3,1,3))
results = model.fit()
results.summary()
forecast_log = results.get_prediction(start=-24).predicted_mean

#forecast for the original data
forecast = np.exp(forecast_log)
forecast

#### Create diagnostic plots

In [ ]:
# Create the 4 diagostics plots
results.plot_diagnostics();
plt.tight_layout()
plt.show();